In [ ]:
# preprocess our data: get the trace and plaintext and labels and save to npz file respectively
# to keep the data format consistent with this repo

In [2]:
import numpy as np
%load_ext autotime

## AES 

In [17]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,  # 0
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,  # 1
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,  # 2
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,  # 3
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,  # 4
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,  # 5
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,  # 6
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,  # 7
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,  # 8
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,  # 9
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,  # a
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,  # b
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,  # c
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,  # d
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,  # e
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16  # f
]


def aes_internal(inp_data_byte, key_byte):
    """
    This function performs XOR operation between the input byte and key byte which is used as label.
    """
#     print(f"inp_data_byte {inp_data_byte},key_byte {inp_data_byte}")
    return sbox[inp_data_byte ^ key_byte]

time: 590 µs


## functions for gen labels

In [4]:
def gen_features_and_labels_256_2(data, key_byte_value, input_target_byte, start_index, end_index):
    """
    This function generates features and labels for the dataset.
    """

    # loading the dataset
    if 'key' in data.files:
        power_traces, plain_text, key = data['power_trace'], data['plain_text'], data['key']
    else:
        power_traces, plain_text = data['power_trace'], data['plain_text']

    print('generating features and labels for the key byte value: ', key_byte_value)

    # key byte is the value between 0 and 15
    labels = []
    for i in range(plain_text.shape[0]):
        text_i = plain_text[i]
        label = aes_internal(text_i[input_target_byte], key_byte_value) #key[i][input_key_byte]
        labels.append(label)

    labels = np.array(labels)
    if not isinstance(power_traces, np.ndarray):
        power_traces = np.array(power_traces)
    power_traces = power_traces[:, start_index:end_index]

    return power_traces, labels


def data_info(data):
    """
    This function prints the information of the dataset.
    """
    # loading the dataset
    power_traces, plain_text, key = data['power_trace'], data['plain_text'], data['key']
    
    print(f'index of data {data.files}')
    print('shape of the power traces: ', power_traces.shape)
    print('shape of the plaintext : ', plain_text.shape)
    print('Ground Truth for the key : ', key)

time: 54.4 ms


## parameters

In [5]:
# parameters of the training dataset
train_data_params = {
    "target_byte": 0,  # byte on which attack is to be performed (at this point, 0 for all unmasked XMEGA)
    "start_idx": 1800,
    "end_idx": 2800,
}

time: 58.2 ms


## process and save attack data in device01 (for testing)

In [18]:
## load data and print info
attack_device01 = np.load('./our_data/unmasked/device01/PC1_CB1_TDX1_K0_U_20k_0317.npz')
data_info(attack_device01)

## get labels 
key_byte_value=attack_device01['key'][train_data_params['target_byte']]
print(f"real key: {key_byte_value}")
attack_power_traces_processed_d1,attack_labels_d1 = gen_features_and_labels_256_2(attack_device01, 
                                                              key_byte_value,
                                                              train_data_params['target_byte'],
                                                              train_data_params['start_idx'],
                                                              train_data_params['end_idx'])

## print processed data info
print(f"processed power traces shape is {attack_power_traces_processed_d1.shape}, data type is {type(attack_power_traces_processed_d1)}")
print(f"generated labels shape is {attack_labels_d1.shape}, number of class {max(attack_labels_d1+1)}, data type is {type(attack_labels_d1)}")
# print(f"sample of processed power traces {attack_power_traces_processed_d1[:3,:]}")


## save power trace, labels, plaintext as npz
# attack_trace_output_path_d1 = './our_data/device01/X_attack.npy'
# attack_plaintext_output_path_d1 = './our_data/device01/plaintexts_attack.npy'
# attack_labels_output_path_d1 = './our_data/device01/Y_attack.npy'
# np.save(attack_trace_output_path_d1,attack_power_traces_processed_d1)
# np.save(attack_plaintext_output_path_d1,attack_device01['plain_text'][:,train_data_params['target_byte']])
# np.save(attack_labels_output_path_d1,attack_labels_d1)
# print('files saved!')

index of data ['power_trace', 'plain_text', 'key']
shape of the power traces:  (40000, 5000)
shape of the plaintext :  (40000, 16)
Ground Truth for the key :  [ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]
real key: 43
generating features and labels for the key byte value:  43
inp_data_byte 232,key_byte 232
inp_data_byte 60,key_byte 60
inp_data_byte 238,key_byte 238
inp_data_byte 145,key_byte 145
inp_data_byte 192,key_byte 192
inp_data_byte 125,key_byte 125
inp_data_byte 166,key_byte 166
inp_data_byte 197,key_byte 197
inp_data_byte 101,key_byte 101
inp_data_byte 18,key_byte 18
inp_data_byte 94,key_byte 94
inp_data_byte 245,key_byte 245
inp_data_byte 5,key_byte 5
inp_data_byte 195,key_byte 195
inp_data_byte 117,key_byte 117
inp_data_byte 158,key_byte 158
inp_data_byte 16,key_byte 16
inp_data_byte 170,key_byte 170
inp_data_byte 129,key_byte 129
inp_data_byte 29,key_byte 29
inp_data_byte 87,key_byte 87
inp_data_byte 89,key_byte 89
inp_data_byte 180,key_byte 180
inp_data

## process train data in device01 (for training)

In [7]:
## load data and print info
train_d1 = np.load('./our_data/unmasked/device01/PC1_CB1_TDX1_K0_U_200k_0317.npz')
data_info(train_d1)

## get labels 
key_byte_value=train_d1['key'][train_data_params['target_byte']]
print(f"real key: {key_byte_value}")
train_power_traces_processed_d1,train_labels_d1 = gen_features_and_labels_256_2(train_d1, 
                                                              key_byte_value,
                                                              train_data_params['target_byte'],
                                                              train_data_params['start_idx'],
                                                              train_data_params['end_idx'])

## print processed data info
print(f"processed power traces shape is {train_power_traces_processed_d1.shape}, data type is {type(train_power_traces_processed_d1)}")
print(f"generated labels shape is {train_labels_d1.shape}, number of class {max(train_labels_d1+1)}, data type is {type(train_labels_d1)}")
# print(f"sample of processed power traces {train_power_traces_processed_d1[:3,:]}")


## save power trace, labels, plaintext as npz
train_trace_output_path_d1 = './our_data/device01/X_train.npy'
train_plaintext_output_path_d1 = './our_data/device01/plaintexts_train.npy'
train_labels_output_path_d1 = './our_data/device01/Y_train.npy'
np.save(train_trace_output_path_d1,train_power_traces_processed_d1)
np.save(train_plaintext_output_path_d1,train_d1['plain_text'][:,train_data_params['target_byte']])
np.save(train_labels_output_path_d1,train_labels_d1)
print('files saved!')

index of data ['power_trace', 'plain_text', 'key']
shape of the power traces:  (200000, 5000)
shape of the plaintext :  (200000, 16)
Ground Truth for the key :  [ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]
real key: 43
generating features and labels for the key byte value:  43
processed power traces shape is (200000, 1000), data type is <class 'numpy.ndarray'>
generated labels shape is (200000,), number of class 256, data type is <class 'numpy.ndarray'>
files saved!
time: 16 s


## process attack data in device02 (for testing fine-tune model)

In [9]:
## load data and print info
attack_d2 = np.load('./our_data/unmasked/device02/PC2_CB2_TDX3_K3_U_20k_0317.npz')
data_info(attack_d2)

## get labels 
key_byte_value=attack_d2['key'][train_data_params['target_byte']]
print(f"real key: {key_byte_value}")
attack_power_traces_processed_d2,attack_labels_d2 = gen_features_and_labels_256_2(attack_d2, 
                                                              key_byte_value,
                                                              train_data_params['target_byte'],
                                                              train_data_params['start_idx'],
                                                              train_data_params['end_idx'])

## print processed data info
print(f"processed power traces shape is {attack_power_traces_processed_d2.shape}, data type is {type(attack_power_traces_processed_d2)}")
print(f"generated labels shape is {attack_labels_d2.shape}, number of class {max(attack_labels_d2+1)}, data type is {type(attack_labels_d2)}")
# print(f"sample of processed power traces {attack_power_traces_processed_d2[:3,:]}")


## save power trace, labels, plaintext as npz
attack_trace_output_path_d2 = './our_data/device02/X_attack.npy'
attack_plaintext_output_path_d2 = './our_data/device02/plaintexts_attack.npy'
attack_labels_output_path_d2 = './our_data/device02/Y_attack.npy'
np.save(attack_trace_output_path_d2,attack_power_traces_processed_d2)
np.save(attack_plaintext_output_path_d2,attack_d2['plain_text'][:,train_data_params['target_byte']])
np.save(attack_labels_output_path_d2,attack_labels_d2)
print('files saved!')

index of data ['power_trace', 'plain_text', 'key']
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [112  25 236  86 208 121 246 166 121 214 187 133  45  97  96  48]
real key: 112
generating features and labels for the key byte value:  112
processed power traces shape is (20000, 1000), data type is <class 'numpy.ndarray'>
generated labels shape is (20000,), number of class 256, data type is <class 'numpy.ndarray'>
files saved!
time: 2.11 s


## process fine_tuning data in device02 (for fine-tuning)

In [13]:
## load data and print info
tuning_d2 = np.load('./our_data/unmasked/device02/PC2_CB2_TDX3_K3_U_2500_b_0317.npz')
print(f'index of data {tuning_d2.files}')
print('shape of the power traces: ', tuning_d2['power_trace'].shape)
print('shape of the plaintext : ', tuning_d2['plain_text'].shape)
# print('Ground Truth for the key : ', key)

## get trace under attack window
## Even though tunning data does not have 'key' --> can not get the key_byte_value --> can not generate corresponding label
## but it has the same 'key_byte_value' as attack data after fine_tuning in above
key_byte_value=attack_d2['key'][train_data_params['target_byte']]
print(f"real key: {key_byte_value}")
tuning_power_traces_processed_d2,tuning_labels_d2 = gen_features_and_labels_256_2(tuning_d2, 
                                                              key_byte_value,
                                                              train_data_params['target_byte'],
                                                              train_data_params['start_idx'],
                                                              train_data_params['end_idx'])

# tuning_power_traces_processed_d2 = tuning_d2['power_trace'][:, train_data_params['start_idx']:train_data_params['end_idx']]

## print processed data info
print(f"processed power traces shape is {tuning_power_traces_processed_d2.shape}, data type is {type(tuning_power_traces_processed_d2)}")
print(f"shape of labels {tuning_labels_d2.shape}")

## save power trace, labels, plaintext as npz
tuning_trace_output_path_d2 = './our_data/device02/X_tuning.npy'
tuning_plaintext_output_path_d2 = './our_data/device02/plaintexts_tuning.npy'
tuning_labels_output_path_d2 = './our_data/device02/Y_tuning.npy'
np.save(tuning_trace_output_path_d2,tuning_power_traces_processed_d2)
np.save(tuning_plaintext_output_path_d2,tuning_d2['plain_text'][:,train_data_params['target_byte']])
np.save(tuning_labels_output_path_d2,tuning_labels_d2)
print('files saved!')

index of data ['power_trace', 'plain_text']
shape of the power traces:  (2500, 5000)
shape of the plaintext :  (2500, 16)
real key: 112
generating features and labels for the key byte value:  112
processed power traces shape is (2500, 1000), data type is <class 'numpy.ndarray'>
shape of labels (2500,)
files saved!
time: 201 ms


## process identity labels to hamming weight labels

In [11]:
## hamming weight function
def hamming_weight(num: int) -> int:
        weight = 0

        while num:
            weight += 1
            num &= num - 1

        return weight
    
def hm_labels(input_path,output_path):
    labels_identity = np.load(input_path)
    labels_hw = []
    for x in labels_identity:
        labels_hw.append(hamming_weight(x))
    np.save(output_path,np.array(labels_hw))
    print(f"max class is {max(labels_hw)}")
    print(f"hw labels transferred and saved!")

time: 315 µs


In [14]:
## device01
labels_train_d1_path = './our_data/device01/Y_train.npy'
labels_attack_d1_path = './our_data/device01/Y_attack.npy'
labels_train_d1_outpath = './our_data/device01/Y_train_hw.npy'
labels_attack_d1_outpath = './our_data/device01/Y_attack_hw.npy'
## device02
labels_attack_d2_path = './our_data/device02/Y_attack.npy'
labels_attack_d2_outpath = './our_data/device02/Y_attack_hw.npy'
labels_tuning_d2_path = './our_data/device02/Y_tuning.npy'
labels_tuning_d2_outpath = './our_data/device02/Y_tuning_hw.npy'

## transform and save
hm_labels(labels_train_d1_path,labels_train_d1_outpath)
hm_labels(labels_attack_d1_path,labels_attack_d1_outpath)
hm_labels(labels_attack_d2_path,labels_attack_d2_outpath)
hm_labels(labels_tuning_d2_path,labels_tuning_d2_outpath)


max class is 8
hw labels transferred and saved!
max class is 8
hw labels transferred and saved!
max class is 8
hw labels transferred and saved!
max class is 8
hw labels transferred and saved!
time: 262 ms
